# Limpieza de Datos Locales 20190712

In [1]:
import pandas as pd
from utils_functions import  replace_regex, remove_non_ascii, upload_to_bucket

# Importar Datasets

In [2]:
locales_col_names = [
    'id_cadena', 'desc_cadena', 'id_region', 'desc_region', 'idb',
    'desc_local', 'latitud', 'longitud', 'punto_dist'
]

In [3]:
locales_df = pd.read_csv("../data/Locales.txt",
                         encoding="UTF-8",
                         header=None,
                         delimiter='\t',
                         names=locales_col_names)

In [4]:
locales_df.head(40)


,id_cadena,desc_cadena,id_region,desc_region,idb,desc_local,latitud,longitud,punto_dist
0,1,Plaza Vea,1,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,1,Plaza Vea,1,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
2,1,Plaza Vea,1,Buenos Aires,9695,695 - P Vea Acoyte.,0.000000,0.000000,N
3,1,Plaza Vea,1,Buenos Aires,9699,699 - P Vea Rivadavia.,0.000000,0.000000,N
4,1,Plaza Vea,1,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N
5,2,Disco,1,Buenos Aires,247,916 - SM 916 Gorostiaga,-34.565259,-58.436583,N
6,2,Disco,1,Buenos Aires,257,61 - Tortuguitas,-34.446043,-58.745098,N
7,2,Disco,1,Buenos Aires,285,953 - Venado Tuerto,NaN,NaN,N
8,2,Disco,1,Buenos Aires,740,740 - Canning,-34.853221,-58.502247,N
9,2,Disco,1,Buenos Aires,961,961 - Av. Santa FÇ,NaN,NaN,N


In [5]:
locales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_cadena    175 non-null    int64  
 1   desc_cadena  175 non-null    object 
 2   id_region    175 non-null    int64  
 3   desc_region  175 non-null    object 
 4   idb          175 non-null    int64  
 5   desc_local   175 non-null    object 
 6   latitud      140 non-null    float64
 7   longitud     140 non-null    float64
 8   punto_dist   175 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 12.4+ KB


#### Ajustes al dataframe

- Limpiar la descripcion de la región, eliminando el numero y el simbolo (-).
- Convertir a minusculas todos los tipos de datos object o string.
- Remplazar caracteres especiales con espacio.
- Verificar si existen valores nulos o duplicados.
- Convertir a 1 y 0 la columna punto de distribucción con la siguiente regla si el valor es 's' = 1 que significa que es un punto de distribucción de lo contrario sería 0.
- Crear una nueva columna llamada local usando la columna punto de distribucción.
- Rellenar los valores de latitud y longitud con valores Nan con la media de la columna de latitud y longitud.
- Buscar los valores de latitud y longitud de regiones con valores 0.00


In [6]:
locales_df = locales_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [7]:
replace_regex(locales_df, 'desc_local', r'^\d+\s*-\s*', '')

In [8]:
replace_regex(locales_df, 'desc_local', r'f', 'fé')
replace_regex(locales_df, 'desc_local', r'crdoba', 'cordoba')
replace_regex(locales_df, 'desc_local', r'fééernando', 'fernando')
replace_regex(locales_df, 'desc_local', r'\bjumbo\b', '')
replace_regex(locales_df, 'desc_local', r'\brincn\b', '')
replace_regex(locales_df, 'desc_local', r'\bplaza vea\b', '')
replace_regex(locales_df, 'desc_local', r'\bp vea\b', '')

In [9]:
remove_non_ascii(locales_df, 'desc_local')
#locales_df['desc_local'] = locales_df['desc_local'].replace(to_replace=r'[^\x00-\x7F]+', value='', regex=True)

In [10]:
print(locales_df.isna().sum())

id_cadena       0
desc_cadena     0
id_region       0
desc_region     0
idb             0
desc_local      0
latitud        35
longitud       35
punto_dist      0
dtype: int64


In [11]:
locales_df.isnull().sum(axis=0)

id_cadena       0
desc_cadena     0
id_region       0
desc_region     0
idb             0
desc_local      0
latitud        35
longitud       35
punto_dist      0
dtype: int64

In [12]:
# Conteo de los niveles en las diferentes columnas categóricas
col_categoricas = ['desc_cadena', 'desc_region', 'desc_local']
for col in col_categoricas:
    print(f'Columna {col}: {locales_df[col].nunique()} subnivele(s)')

Columna desc_cadena: 5 subnivele(s)
Columna desc_region: 1 subnivele(s)
Columna desc_local: 175 subnivele(s)


In [13]:
# Convertir la columna categorica punto_dist a numerica, si el valor es s significa que es un punto de distribucción y se asignará el valor de 1, de lo contrario será  un local y se asignara el valor de 0.
punto_dist_mapping = {"s": 1, "n": 0}
locales_df['punto_dist'] = locales_df['punto_dist'].map(punto_dist_mapping)

In [14]:
# Crear una columna para determinar si es un local con base en la columna punto_dist, si el local no es un punto de distribuccion entonces es un local.  Asignaremos 1 para local 0 para no local.
locales_df['local'] = locales_df['punto_dist'].apply(lambda x: 1 if x == 0 else 0)

In [15]:
# Llenar los valores missing de latitud y longitud con la media de cada columna.
mean_latitud = locales_df['latitud'].mean()
mean_longitud = locales_df['longitud'].mean()
locales_df['latitud'].fillna(mean_latitud, inplace=True)
locales_df['longitud'].fillna(mean_longitud, inplace=True)

In [16]:
locales_df.head(200)

,id_cadena,desc_cadena,id_region,desc_region,idb,desc_local,latitud,longitud,punto_dist,local
0,1,plaza vea,1,buenos aires,9693,villa urquiza.,-31.533832,-52.839588,0,1
1,1,plaza vea,1,buenos aires,9694,jb justo.,-31.533832,-52.839588,0,1
2,1,plaza vea,1,buenos aires,9695,acoyte.,0.000000,0.000000,0,1
3,1,plaza vea,1,buenos aires,9699,rivadavia.,0.000000,0.000000,0,1
4,1,plaza vea,1,buenos aires,9889,carpa de liqui,-31.533832,-52.839588,0,1
...,...,...,...,...,...,...,...,...,...,...
170,11,rincon jumbo,1,buenos aires,8306,rincn palermo,-31.533832,-52.839588,0,1
171,11,rincon jumbo,1,buenos aires,8311,rincn pilar,0.000000,0.000000,0,1
172,11,rincon jumbo,1,buenos aires,8314,rincn escobar,-31.533832,-52.839588,0,1
173,11,rincon jumbo,1,buenos aires,8318,rincn rosario,-31.533832,-52.839588,0,1


In [17]:
locales_df.describe()

,id_cadena,id_region,idb,latitud,longitud,punto_dist,local
count,175.000000,175.0,175.000000,175.000000,175.000000,175.000000,175.000000
mean,4.502857,1.0,6909.617143,-31.533832,-52.839588,0.034286,0.965714
std,2.945627,0.0,3421.539461,9.580378,15.841742,0.182484,0.182484
min,1.000000,1.0,242.000000,-45.872649,-67.496036,0.000000,0.000000
25%,2.000000,1.0,5213.500000,-34.660845,-58.535721,0.000000,1.000000
50%,4.000000,1.0,9019.000000,-34.583457,-58.404329,0.000000,1.000000
75%,4.000000,1.0,9158.500000,-31.533832,-52.839588,0.000000,1.000000
max,11.000000,1.0,9977.000000,0.000000,0.000000,1.000000,1.000000


#### Observaciones




In [18]:
import folium

# Combine the latitude and longitude columns into a list of locations
locations = [[lat, lon] for lat, lon in zip(locales_df['latitud'], locales_df['longitud'])]

# Create a map centered at the first location in the list
m = folium.Map(location=locations[0], zoom_start=5)

# Add markers to the map for each location in the list
for location in locations:
    folium.Marker(location=location).add_to(m)

# Display the map
m

#### Guardar el dataset limpio

In [19]:
locales_df.to_csv('../clean_data/locales_clean.csv', index=False)

#### Subir csv a google cloud storage

In [20]:
upload_to_bucket('deodorants', '../clean_data/locales_clean.csv', 'locales')

File ../clean_data/locales_clean.csv uploaded successfully to bucket deodorants/locales
